In [1]:
import numpy as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
%matplotlib inline

from zipline.data import bundles

In [11]:
from sklearn.decomposition import PCA


def get_num_components(df, var_ret):
    '''
    Given an n-dimensional data set, identify the number of principal 
    components needed to explain the amount of retained variance
    
    '''    
    if var_ret > 1 or var_ret < 0:
        print('Error')
        return 0
    
    if var_ret == 1:
        return df.shape[1]
        
    pca = PCA(n_components = df.shape[1])

    pca.fit(df)
    
    needed_components = 0
    
    var_sum = 0
    
    for i in range(0, df.shape[1]):

        if var_sum >= var_ret:
            print('Total Variance Retained: ', pca.explained_variance_ratio_[0:needed_components].sum())
            return needed_components
        else:
            needed_components += 1            
            var_sum += pca.explained_variance_ratio_[i]
    return needed_components
        
            


In [17]:

def factor_betas(pca, factor_beta_indices, factor_beta_columns):
    '''
    Constructs new dataframe with pca object principal components
    in the original basis coordinate system
    
    Parameters
    ==========
    pca: 
        sklearn.decomposition.PCA object fitted with returns df
    factor_beta_indices: 
        1-D np.ndarray containing column names of the returns dataframe
    factor_beta_columns: 
        1-d ndarray containing evenly spaced integers from 0 up to # principial components minus 1
        
        
    Returns
    =======
    
    pd.Dataframe with factor_beta_indices as rows and factor_beta_columns as columns
    
    '''
        
    return pd.DataFrame(
                        data=pca.components_.T, 
                        index=factor_beta_indices, 
                        columns=factor_beta_columns
                        )
    


In [18]:
import pandas as pd 

def factor_returns(pca, returns, factor_return_indices, factor_return_columns):
    '''
    Uses sklearn pca.transform to project returns into the new basis. 
    Returns the factor returns for in the new basis.
    
    Parameters
    ==========
    pca: 
        sklearn.decomposition.PCA object fitted with returns df
    factor_return_indices: 
        1-D np.ndarray containing index of trading dates factor returns dataframe
    factor_return_columns: 
        1-d ndarray containing evenly spaced integers from 0 up to # principial components minus 1
        
        
    Returns
    =======
    
    pd.Dataframe of returns in the new basis 
        with factor_return_indices as rows and factor_return_columns as columns
    
    '''
    
    return pd.DataFrame(
                        data=pca.transform(returns), 
                        index=factor_return_indices,
                        columns=factor_return_columns
                        )


In [19]:
def idiosyncratic_var_matrix(returns, factor_returns, factor_betas, ann_factor):
    '''
    Our matrix of residuals that represents the difference of the observed returns minus
    the product of factor returns matrix and beta factors matrix. 
    
    Parameters
    ==========
    returns: pd.DataFrame of asset returns of shape N companies by M time steps
    factor_returns: pd.DataFrame of asset returns in the new basis
    factor_betas: pd.DataFrame of betas for the principal components
    ann_factor: annualization factor
    
    Returns
    =======
    pd.DataFrame of residuals with the 
        annualized Idiosyncratic Risk Matrix containing the 
        covariance of the residuals in its main diagonal
    
    '''
    common_returns_ = pd.DataFrame(np.dot(factor_returns, factor_betas.T), returns.index, returns.columns)
    
    residuals_ = (returns - common_returns_)
    
    return pd.DataFrame(np.diag(np.var(residuals_))*ann_factor, returns.columns, returns.columns)
    


In [20]:
def factor_cov_matrix(factor_returns, ann_factor):
    '''
    Calculates the diagonal annualized factor covariance Matrix
    
    Parameters
    ==========
    factor_returns: pd.DataFrame of new basis
    ann_factor: np.float
    
    Returns
    =======
    np.ndarray of annualized factor covariance
    
    '''
    return np.diag(factor_returns.var(axis=0, ddof=1)*ann_factor)
 
    
    


In [21]:
from sklearn.decomposition import PCA

def fit_pca(returns, num_factor_exposures, svd_solver):
    '''
    Wrapper for sklearn PCA class
    
    Parameters
    ==========
    returns: pd.DataFrame
    num_factor_exposures: int
    svd_solver: string for 
    
    Returns
    =======
    sklearn.decomposition.PCA object fitted with the returns values
    '''
    pca = PCA(
            n_components=num_factor_exposures, 
            svd_solver=svd_solver)
    
    pca.fit(returns)
    
    return pca
    


In [23]:
import numpy as np

class RiskModel(object):
    
    def __init__(self, returns, ann_factor, num_factor_expenses, pca):
        
        self.factor_betas_ = factor_betas(pca, returns.columns.values, np.arange(num_factor_exposures))
        
        self.factor_returns_ = factor_returns(pca, returns, returns.index, np.arange(num_factor_exposures))
        
        self.factor_cov_matrix_ = factor_cov_matrix(self.factor_returns_, ann_factor)
        
        self.idiosyncratic_var_matrix_ = idiosyncratic_var_matrix(returns, self.factor_returns_, self.factor_betas_, ann_factor)
        

# # Set the annualized factor
# ann_factor = 252

# # Set the number of factor exposures (principal components) for the PCA algorithm
# num_factor_exposures = 10

# # Set the svd solver for the PCA algorithm
# svd_solver = 'full'

# # Fit the PCA Model using the fit_pca() fucntion 
# pca = fit_pca(returns,num_factor_exposures, svd_solver)

# # Create a RiskModel object
# rm = RiskModel(returns, ann_factor, num_factor_exposures, pca)

In [24]:
## Factor Exposures

#rm.factor_betas_

In [26]:
## Factor Returns

##rm.factor_returns_

In [25]:
##Idiosyncratic Risk Matrix

##rm.idiosyncratic_var_matrix_ 

In [27]:
## factor cov matrix
#rm.factor_cov_matrix_

In [ ]:
#momentum_amplified_mean_reversion strategy
#algorithm definition

#for each security, compute the percentage of time the daily price has gone above/below 20 day ema and stayed there for 
#another 30 days, 80 days, 120 days.

#Daily returns

#compute the 20 day, 50 day, 100 day and 200 day exponential moving averages for a series of securities
#Everytime price crosses 20-day ema from below/above increment. If price still above/below 20-day ema 3 days later. Buy/sell
#If price crosses 50-day ema and stays above for 3 days add to position. 



In [28]:
#%matplotlib inline

# import matplotlib.pyplot as plt

# # Set the default figure size
# plt.rcParams['figure.figsize'] = [10.0, 6.0]

# # Make the bar plot
# plt.bar(np.arange(num_factor_exposures), pca.explained_variance_ratio_);

In [ ]:
# %matplotlib inline

# import matplotlib.pyplot as plt

# # Set the default figure size
# plt.rcParams['figure.figsize'] = [10.0, 6.0]

# rm.factor_returns_.loc[:,0:5].cumsum().plot();